# Merging Data Gemastik

Data yang akan disatukan didapat dari BPS , berikut adalah data yang dipakai :


| No  | Nama Data                                                                             | Nama Kolom                                                          | Sumber Data                                                                                                                                           |
|-----|---------------------------------------------------------------------------------------|---------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1.  | PDRB (Harga Konstan)                                                                  | PDRB                                                                | https://www.bps.go.id/indicator/52/286/1/-seri-2010-produk-domestik-regional-bruto-.html                                                              |
| 2.  | Persentase penduduk miskin                                                            | persentase kemiskinan                                               | https://www.bps.go.id/indicator/23/192/1/persentase-penduduk-miskin-p0-menurut-provinsi-dan-daerah.html                                               |
| 3.  | Persentase jumlah siswa SMA yang lulus                                                | persentase_lulus_sma                                                | https://www.bps.go.id/indicator/28/1980/1/tingkat-penyelesaian-pendidikan-menurut-jenjang-pendidikan-dan-provinsi.html                                |
| 4.  | Tingkat pengangguran terbuka                                                          | persentase_pengangguran                                             | https://www.bps.go.id/indicator/6/543/2/tingkat-pengangguran-terbuka-menurut-provinsi.html                                                            |
| 5.  | Angka harapan lama sekolah                                                            | harapan_lama_sekolah                                                | https://www.bps.go.id/indicator/26/415/1/-metode-baru-rata-rata-lama-sekolah.html                                                                     |
| 6   | Pengeluaran perkapita                                                                 | pengeluaran_perkapita                                               | https://www.bps.go.id/indicator/26/416/1/-metode-baru-pengeluaran-per-kapita-disesuaikan.html                                                         |
| 7.  | Angka harapan hidup                                                                   | angka_harapan_hidup                                                 | https://www.bps.go.id/indicator/26/414/1/-metode-baru-umur-harapan-hidup-saat-lahir-uhh-.html                                                         |
| 8.  | Proporsi Rumah Tangga Yang Memiliki Akses Terhadap Layanan Sanitasi Layak             | persentase_sanitasi_layak                                           | https://www.bps.go.id/indicator/29/1267/1/proporsi-rumah-tangga-yang-memiliki-akses-terhadap-layanan-sanitasi-layak.html                              |
| 9   | Persentase Rumah Tangga yang Memiliki Akses Terhadap Hunian Yang Layak Dan Terjangkau | persentase_rumah_layak                                              | https://www.bps.go.id/indicator/29/1241/1/persentase-rumah-tangga-yang-memiliki-akses-terhadap-hunian-yang-layak-dan-terjangkau-menurut-provinsi.html |
| 10. | Jumlah Penduduk                                                                       | penduduk_ribu_jiwa                                                  | https://sulut.bps.go.id/indicator/12/958/1/jumlah-penduduk-menurut-provinsi-di-indonesia.html ,  https://www.bps.go.id/indicator/12/1886/1/jumlah-penduduk-hasil-proyeksi-menurut-provinsi-dan-jenis-kelamin.html                                                        |
| 11. | Panjang Jalan Menurut Provinsi dan Tingkat Kewenangan Pemerintahan                    | jalan negara, jalan provinsi, jalan kabupaten /kota, total panjang  | https://bps.go.id/indikator/indikator/view_data_pub/0000/api_pub/SENxVDdSbVprRGF2Sk91UTdGcVdTdz09/da_10/1                                             |
| 12. | Persentase Rumah Tangga menurut Provinsi, Tipe Daerah dan Sumber Air Minum Layak      | persentase_air_bersih                                               | https://www.bps.go.id/indicator/29/854/1/persentase-rumah-tangga-menurut-provinsi-tipe-daerah-dan-sumber-air-minum-layak.html                         |

Notes : 
 -  Tingkat pengangguran terbuka dihitung berdasarkan rata - rata pengangguran dari bulan Februari sampai Oktober 
 -  Tingkat persentase kemiskinan dihitung berdasarkan rata - rata persentase dari bulan Maret sampai September 
 - Karena keterbatasan data, panjang jalan di tahun 2015 dihitung dengan ekstrapolasi panjang jalan dari tahun 2016 - 2022

In [99]:
import pandas as pd
import numpy as np

In [100]:
def lower_prov(df):
    df.Provinsi = df.Provinsi.str.lower()
    return df

def make_df(location,name):
    df = lower_prov(pd.read_excel(location))
    end_year = 2031 if name == "penduduk_ribu_jiwa" else 2023
    li_col = ['Provinsi']
    li_year = [x for x in range(2015,end_year)]
    li_col.extend(li_year)
    df = df[li_col]
    df = df.melt(id_vars=['Provinsi'], value_vars=li_col)
    df = df.rename(columns = {'variable':'tahun','value':name}).sort_values(by = ['Provinsi','tahun'])
    return df

# Variabel Yang Dipakai

In [101]:

dataset = [
    ['sourceData\\pdrb.xlsx','PDRB'],
    ['sourceData\\sanitasi.xlsx','persentase_sanitasi_layak'],
    ['sourceData\\pengangguran.xlsx','persentase_pengangguran'],
    ['sourceData\\lulus_sma.xlsx','persentase_lulus_sma'],
    ['sourceData\\panjang_jalan.xlsx','panjang_jalan'],
#     ['sourceData\\new_total_penduduk.xlsx','penduduk_ribu_jiwa'],
    ['sourceData\\harapan_lama_sekolah.xlsx','harapan_lama_sekolah'],
    ['sourceData\\air_minum.xlsx','persentase_air_bersih'],
    ['sourceData\\rumah_layak.xlsx','persentase_rumah_layak'],
    ['sourceData\\angka_harapan_hidup.xlsx','angka_harapan_hidup'],
    ['sourceData\\pengeluaran.xlsx','pengeluaran_perkapita'],
    ['sourceData\\average_persentase penduduk_miskin.xlsx','persentase penduduk_miskin'],
]

df_merged = None

for location,name in dataset:
    df_temp = None
    if name == 'panjang_jalan':
        df_temp = lower_prov(pd.read_excel(location)).sort_values(by = ['Provinsi','tahun'])
        df_temp = df_temp.interpolate(method = 'linear',limit_direction = 'backward')
    else:
        df_temp = make_df(location,name)
        if name == 'PDRB':
            df_merged = df_temp
            continue

    df_merged = df_merged.merge(df_temp, how = 'left', on = ['Provinsi','tahun'])
    
# 

# Merged Dataset

In [103]:
penduduk_ribu_jiwa = make_df('sourceData\\new_total_penduduk.xlsx','penduduk_ribu_jiwa')
df_merged = penduduk_ribu_jiwa.merge(df_merged , on = ['Provinsi',"tahun"] , how = "left")
df_merged.tahun = pd.to_datetime(df_merged.tahun,format='%Y')

In [104]:
df_merged.Provinsi.unique()

array(['aceh', 'bali', 'banten', 'bengkulu', 'di yogyakarta',
       'dki jakarta', 'gorontalo', 'jambi', 'jawa barat', 'jawa tengah',
       'jawa timur', 'kalimantan barat', 'kalimantan selatan',
       'kalimantan tengah', 'kalimantan timur', 'kalimantan utara',
       'kepulauan bangka belitung', 'kepulauan riau', 'lampung', 'maluku',
       'maluku utara', 'nusa tenggara barat', 'nusa tenggara timur',
       'papua', 'papua barat', 'riau', 'sulawesi barat',
       'sulawesi selatan', 'sulawesi tengah', 'sulawesi tenggara',
       'sulawesi utara', 'sumatera barat', 'sumatera selatan',
       'sumatera utara'], dtype=object)

In [105]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 543
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Provinsi                    544 non-null    object        
 1   tahun                       544 non-null    datetime64[ns]
 2   penduduk_ribu_jiwa          544 non-null    int64         
 3   PDRB                        272 non-null    float64       
 4   persentase_sanitasi_layak   272 non-null    float64       
 5   persentase_pengangguran     272 non-null    float64       
 6   persentase_lulus_sma        272 non-null    float64       
 7   jalan_negara                272 non-null    float64       
 8   jalan_provinsi              272 non-null    float64       
 9   jalan_kabupaten_kota        272 non-null    float64       
 10  panjang_total               272 non-null    float64       
 11  harapan_lama_sekolah        272 non-null    float64       

In [106]:
df_merged.isna().sum()

Provinsi                        0
tahun                           0
penduduk_ribu_jiwa              0
PDRB                          272
persentase_sanitasi_layak     272
persentase_pengangguran       272
persentase_lulus_sma          272
jalan_negara                  272
jalan_provinsi                272
jalan_kabupaten_kota          272
panjang_total                 272
harapan_lama_sekolah          272
persentase_air_bersih         272
persentase_rumah_layak        272
angka_harapan_hidup           272
pengeluaran_perkapita         272
persentase penduduk_miskin    272
dtype: int64

In [107]:
df_merged[df_merged["harapan_lama_sekolah"].isna()].tahun.unique()

array(['2023-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2025-01-01T00:00:00.000000000', '2026-01-01T00:00:00.000000000',
       '2027-01-01T00:00:00.000000000', '2028-01-01T00:00:00.000000000',
       '2029-01-01T00:00:00.000000000', '2030-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [108]:
df_merged[df_merged.Provinsi == 'dki jakarta']

,Provinsi,tahun,penduduk_ribu_jiwa,PDRB,persentase_sanitasi_layak,persentase_pengangguran,persentase_lulus_sma,jalan_negara,jalan_provinsi,jalan_kabupaten_kota,panjang_total,harapan_lama_sekolah,persentase_air_bersih,persentase_rumah_layak,angka_harapan_hidup,pengeluaran_perkapita,persentase penduduk_miskin
80,dki jakarta,2015-01-01,10178942,1989088.75,88.39,7.795,74.10,180.0,3900.5,1691.0,5771.5,12.59,93.40,99.25,72.43,17075.0,3.770
81,dki jakarta,2016-01-01,10265267,2159073.60,90.42,5.945,74.74,53.0,7041.0,0.0,7094.0,12.73,92.44,99.49,72.49,17468.0,3.750
82,dki jakarta,2017-01-01,10348331,2365353.85,90.37,6.250,78.25,53.0,6679.0,0.0,6732.0,12.86,88.93,99.49,72.55,17707.0,3.775
83,dki jakarta,2018-01-01,10427981,2592606.57,90.73,6.190,83.48,53.0,6679.0,0.0,6732.0,12.95,89.59,99.36,72.67,18128.0,3.560
84,dki jakarta,2019-01-01,10504035,2815636.16,92.89,6.020,84.35,53.0,6432.0,0.0,6485.0,12.97,99.82,34.25,72.79,18527.0,3.445
85,dki jakarta,2020-01-01,10576393,2767273.49,93.04,8.050,85.67,53.0,6432.0,0.0,6485.0,12.98,99.84,33.18,72.91,18227.0,4.610
86,dki jakarta,2021-01-01,10645513,2912563.13,95.17,8.505,84.98,53.0,6432.0,0.0,6485.0,13.07,99.86,40.00,73.01,18520.0,4.695
87,dki jakarta,2022-01-01,10711777,3186469.91,92.79,7.590,87.71,53.0,0.0,6432.0,6485.0,13.08,97.93,36.23,73.32,18927.0,4.650
88,dki jakarta,2023-01-01,10775046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,dki jakarta,2024-01-01,10835111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df_merged.isna().sum()

Provinsi                        0
tahun                           0
penduduk_ribu_jiwa              0
PDRB                          272
persentase_sanitasi_layak     272
persentase_pengangguran       272
persentase_lulus_sma          272
jalan_negara                  272
jalan_provinsi                272
jalan_kabupaten_kota          272
panjang_total                 272
harapan_lama_sekolah          272
persentase_air_bersih         272
persentase_rumah_layak        272
angka_harapan_hidup           272
pengeluaran_perkapita         272
persentase penduduk_miskin    272
dtype: int64

In [111]:
df_merged.to_csv("merged_data.csv")